In [14]:
from artistplaylist import ArtistPlaylist
import event_scraper
import pprint

events = event_scraper.get_events(1, 10)
print("Found {0} events".format(len(events)))

Found 10 events


In [15]:
for ev in events:
    print(ev['artist'])

Autograf and Ayokay
Angelo Moore and the Brand New Step
Charlotte DJ Set
Lil Mike & Funny Bone, Lil Mike & FunnyBone
Carter Arrington
Young and Rusty
Tiarra Girls, Grupo Fantasma, Bombasta and 3 more...
Teddy Long Music
Werefoot, Ayasoltec, Toulca and 6 more...
Oscar Ornelas


In [17]:
ap = ArtistPlaylist()

song_ids = []
for ev in events:
    songs = ap.songs_from_artist(ev['artist'])
    result = None
    sid = None
    if songs:
        sid = songs[0]['storeId']
        song_ids.append(sid)
        result = songs[0]['genre']
    print(' -- '.join([ev['artist'], str(sid) , str(result)]))

INFO:gmusicapi.Mobileclient10:initialized
INFO:gmusicapi.Mobileclient10:logged out
INFO:gmusicapi.Mobileclient10:authenticated
INFO:gmusicapi.Mobileclient10:subscribed


Autograf and Ayokay -- None -- None
Angelo Moore and the Brand New Step -- T2vp2yooyjrcim5emvwhi3rmnm4 -- Jazz
Charlotte DJ Set -- None -- None
Lil Mike & Funny Bone, Lil Mike & FunnyBone -- Tltlsyiway6ahed42y57ylpjv7u -- Hip-Hop/Rap
Carter Arrington -- Tsdokfkhbxoddydojsej4vikq4m -- Hip-Hop/Rap
Young and Rusty -- T5k5buy62iqgsv5lvqtk4o35nwy -- Vocal/Easy Listening
Tiarra Girls, Grupo Fantasma, Bombasta and 3 more... -- None -- None
Teddy Long Music -- T3g3r74to6xokh2hxclb2mccfsq -- Pop
Werefoot, Ayasoltec, Toulca and 6 more... -- None -- None
Oscar Ornelas -- None -- None


In [18]:
ap.create_playlist(song_ids, 'ArtistPlaylistDemo')